In [10]:
using JuMP
using CPLEX
using LinearAlgebra

Now we define the master problem :)

In [5]:
function solve_master(A,b,c, op_A, op_B, feas_A, feas_B)
    nx = size(c, 2)
    m = Model(CPLEX.Optimizer)
    if size(op_A, 1) == 0
        @variable(m, x[1:nx])
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx))
        @constraint(m, A*x .<= b)
        optimize!(m)
        theta = -Inf
    else
        @variable(m, x[1:nx])
        @variable(m, theta)
        @objective(m, Min, sum(c[i] * x[i] for i = 1:nx) + theta)
        @constraint(m, A*x .<= b)
        for i = 1:size(op_A, 1)
            @constraint(m, 
                sum(op_A[i,j] * x[j] for j = 1:nx) + op_A[i,nx+1] * theta .>= op_B[i])
        end
        optimize!(m)
        theta = JuMP.value(theta)
    end
    sol = zeros(1, nx)
    for i = 1:nx
            sol[i] = JuMP.value(x[i])
    end
    sol = vec(sol)
    return (objective_value(m), sol, theta, JuMP.termination_status(m))
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[5]:5

In [6]:
function step2(W, h, T, x, omega, q)
    println("Beginning step 2 ...")
    m = Model(CPLEX.Optimizer)
    ny = size(W,2)
    
    @variable(m, y[1:ny] >= 0)

    @constraint(m,constraint, W*y - h[:,omega] + T*x .<= 0)
    @objective(m, Min, transpose(q[:,omega]) * y)
    optimize!(m);
    sol = zeros(1, ny)
    for i = 1:ny
        sol[i] = JuMP.value(y[i])
    end
    sol = vec(sol)
    return (objective_value(m), sol, dual.(constraint), JuMP.termination_status(m))
    println("Step 2 Fini :)")
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[6]:6

In [7]:
function main(A, b, c, T, W, h, q, p)
    V = []
    S = []
    
    k = 0
    val_k, x_k, theta_k = 0, 0, 0

    while (k<10000000000)
        println(k)
        
        (val_k, x_k, theta_k, termination_status) = solve_master(A, b, c, h, T, V, S)
    
        if termination_status == MOI.INFEASIBLE
            return (-1, -1, -1)
        end
        
        V_temp = []
        status = 0
        
        for w in 1:size(p,2)
            print("w : ")
            println(w)
            (pi_k, y_k, sigma_k, termination_status) = step2(W, h, T, x_k, w, q)
            
            if termination_status == MOI.INFEASIBLE
                println("Infeasible step 2")
                S_k = [S_k; sigma_k]
                k += 1
                status = 1
                break
            else
                println("Feasible step 2")
                if V_temp == []
                    V_temp = [p[w]*pi_k]
                else
                    V_temp = [V_temp p[w]*pi_k]
                end
            end
        end
        if(status == 0)
            print("Added vector : ")
                println(V_temp)
            if V == []
                V_new  = V_temp
            else
                V_new = [V; V_temp]
            end
            
            if(V_new == V)
                return (x_k, y_k, k)
            else
                k += 1
                V = V_new
            end
        end
    end
end 

main (generic function with 1 method)

In [8]:
d = [500 100; 300 300]
q = [-24 -28; -28 -32]
p = [0.4 0.6]

A = [1 1; -1 0; 0 -1]
b = [120; -40; -20]
c = [100; 150]

T = [-60 0; 0 -80; 0 0; 0 0]
W = [6 10; 8 5; 1 0; 0 1]
h = [0 0; 0 0; d[1,1] d[2,1]; d[1,2] d[2,2]]

(x,y, k) = main(A, b, c, T, W, h, q, p)

println(x)
println(y)
println(k)


0


LoadError: UndefVarError: solve_master not defined